# Using two approaches to solve the question.

#### Approach Decisions
 - After examining the size of the datasets I decided to go with these 2 approaches among numerous other approaches.  I briefly also considered uploading the data into a HIVE table but then decided that I'd rather use Sqlite since the dataset size was small. Although in real world scenarios we could easily have written the data to an S3 location after processing instead of to a SqlLite table.
 - This sort of flat data could also be stored in a MongoDB database after using jsonify libraries to process the Unix MBox file. 
 - I decided to use Pandas for all the data processing so I went with this approach.
 - Currently running on an Anaconda distro of Python.

In [6]:
import operator
import pandas as pd
import email
import os
from time import strftime
from functools import partial
from sqlalchemy import create_engine

import time
from dateutil.parser import parse

Return email as "direct" if there is exactly one recipient and "broadcast" if it has multiple recipients.

In [13]:
def helper_email_categorize(email_line):
    if email_line is not None:
        email_line_list = email_line.split("|")
        email_line_list = filter(lambda k: k != "None", email_line_list)
        if len(email_line_list) > 1:
            return 'broadcast'
        elif len(email_line_list) == 1:
            return 'direct'
        else:
            return None

Removes prefixes like 'FW:' and 'Re:' in the subject of an email.

In [14]:
def helper_remove_subject_prefixes(subject_line, remove_prefix_list=None):
    while subject_line is not None and subject_line[:3] in remove_prefix_list:
        subject_line = subject_line[4:]
    return subject_line

** creating sqlite connection with database name. **

In [15]:
disk_engine = create_engine('sqlite:///slack.db')

In [70]:
all_dir_files = [[os.path.join(root, file_name) for file_name in file_names if ".cats" not in file_name] for
                     (root, dirs, file_names) in
                     os.walk('enron_with_categories') if
                     len(file_names) > 0]
total_files_processed = 0
disk_engine.execute('DROP TABLE IF EXISTS processed_data;')

remove_content_function = partial(helper_remove_subject_prefixes, remove_prefix_list=['Re:', 'RE:'])

Applying preprocessing for the following fields. Date is parsed using dateutil.parser

In [71]:
initial_time = time.clock()
for files in all_dir_files:
    t = time.clock()
    email_files = map(lambda k: email.message_from_file(open(k, 'r')), files)
    df = pd.DataFrame([[email_m['Message-ID'], email_m['To'], email_m['From'], email_m['Cc'], email_m['Bcc'],
                        email_m['Date'], email_m['Subject']] for email_m in email_files],
                      columns=["Message-ID", "To", "From", "Cc", "Bcc", 'Date', 'Subject'])

    df['To'] = df['To'].map(lambda k: k if k is not None else "None")
    df['Cc'] = df['Cc'].map(lambda k: k if k is not None else "None")
    df['Bcc'] = df['Bcc'].map(lambda k: k if k is not None else "None")
    df['Date'] = df['Date'].map(lambda k: strftime("%Y-%m-%d %H:%M:%S", (parse(k).timetuple())))
    df['Day'] = df['Date'].map(lambda k: k.split(" ")[0])
    df['effective_to'] = df[['To', 'Cc', 'Bcc']].values.tolist()
    df['effective_to'] = df['effective_to'].map(lambda k: "|".join(k))
    df['message_type'] = df['effective_to'].map(helper_email_categorize)
    df['Subject'] = df['Subject'].map(remove_content_function)

    df.to_sql('processed_data', disk_engine, if_exists='append')
    print "Completed {} email rows in: {} secs.".format(len(files), (time.clock() - t))
    total_files_processed += len(files)

print "Total files processed {} in {} secs. ".format(total_files_processed, time.clock()-initial_time)

Completed 143 email rows in: 0.147023 secs.
Completed 21 email rows in: 0.051647 secs.
Completed 74 email rows in: 0.094396 secs.
Completed 476 email rows in: 0.358272 secs.
Completed 18 email rows in: 0.043392 secs.
Completed 834 email rows in: 0.826454 secs.
Completed 36 email rows in: 0.041453 secs.
Completed 100 email rows in: 0.100702 secs.
Total files processed 1702 in 1.66605 secs. 


## Starting question 1

### Approach 1
- Read everything into a pandas df.

In [62]:
df = pd.read_sql_query('SELECT * FROM processed_data', disk_engine)

In [63]:
t = time.clock()
df['Day'] = df['Date'].map(lambda k: k.split(" ")[0])
df_qtn1 = pd.DataFrame(df.groupby(['From', 'Day'])['Date'].count())
df_qtn1 = df_qtn1.rename(columns={'Date': 'No of emails received'})
df_qtn1.to_csv('q1_output.csv')
print df_qtn1.head(10)
print "Completed question 1. in %f secs." % (time.clock() - t)

                                  No of emails received
From                  Day                              
40enron@enron.com     2001-08-02                      1
alan.comnes@enron.com 2000-12-04                      1
                      2001-01-26                      1
                      2001-02-03                      1
                      2001-03-14                      1
                      2001-06-06                      2
                      2001-06-07                      1
                      2001-06-22                      1
                      2001-06-23                      1
                      2001-07-25                      1
Completed question 1. in 0.023263 secs.


### Approach 2
- This approach is important if the data is very large or in a HIVE table. We can easily filter the data in the query before loading it into a pandas dataframe.

In [74]:
t = time.clock()
df_qtn1 = pd.read_sql_query('SELECT "From", Day, count(Date) as \'No of emails received\' '
                           'FROM processed_data '
                           'group by "From", Day', disk_engine)

df_qtn1.to_csv('q1_output_approach_2.csv')
print df_qtn1.head(10)
print "Completed question 1. in %f secs." % (time.clock() - t)

                    From         Day  No of emails received
0      40enron@enron.com  2001-08-02                      1
1  alan.comnes@enron.com  2000-12-04                      1
2  alan.comnes@enron.com  2001-01-26                      1
3  alan.comnes@enron.com  2001-02-03                      1
4  alan.comnes@enron.com  2001-03-14                      1
5  alan.comnes@enron.com  2001-06-06                      2
6  alan.comnes@enron.com  2001-06-07                      1
7  alan.comnes@enron.com  2001-06-22                      1
8  alan.comnes@enron.com  2001-06-23                      1
9  alan.comnes@enron.com  2001-07-25                      1
Completed question 1. in 0.037114 secs.


## Starting question 2 part-1.

### Approach 1

In [79]:
t = time.clock()
qtn2_1_df = df[df['message_type'] == 'direct']
qtn2_1_df = qtn2_1_df[['To', 'message_type']]
qtn2_1_df = pd.DataFrame(qtn2_1_df.groupby(['To'], sort=True).count())
qtn2_1_df = qtn2_1_df.sort_values(['message_type'], ascending=False).head(6)
qtn2_1_df = qtn2_1_df.rename(columns={'message_type': 'Number of emails'})
print qtn2_1_df
print "Completed question 1. in: %f secs." % (time.clock()-t)

                           Number of emails
To                                         
vkaminski@aol.com                         4
skean@enron.com                           3
linda.robertson@enron.com                 3
jeff.dasovich@enron.com                   3
charlotte@wptf.org                        2
j.kaminski@enron.com                      2
Completed question 1. in: 0.015271 secs.


### Approach 2

In [80]:
t = time.clock()
qtn2_1_df = pd.read_sql_query('SELECT "To", count(message_type) as \'No of emails\' '
                                  'FROM processed_data '
                                  'where message_type == "direct"'
                                  'group by "To" order by count(message_type) DESC limit 6;', disk_engine)
print qtn2_1_df
print "Completed in: %f secs." % (time.clock()-t)

                             To  No of emails
0    maureen.mcvicker@enron.com           115
1             vkaminski@aol.com            43
2       jeff.dasovich@enron.com            25
3     richard.shapiro@enron.com            23
4   elizabeth.linnell@enron.com            18
5  bernadette.hawkins@enron.com            17
Completed in: 0.021925 secs.


## Starting question 2 part-2.

### Approach 1

In [67]:
t = time.clock()
qtn2_2_df = df[['From', 'message_type']]
qtn2_2_df = pd.DataFrame(qtn2_2_df[qtn2_2_df['message_type'] == 'broadcast'].groupby(['From'], sort=True).count())
qtn2_2_df = qtn2_2_df.sort_values(['message_type'], ascending=False).head(6)
qtn2_2_df = qtn2_2_df.rename(columns={'message_type': 'Number of emails'})
print qtn2_2_df
print "Completed in: %f secs." % (time.clock()-t)

                              Number of emails
From                                          
steven.kean@enron.com                      175
j.kaminski@enron.com                        41
john.shelk@enron.com                        36
alan.comnes@enron.com                       13
kevinscott@onlinemailbox.net                11
michelle.cash@enron.com                     10
Completed in: 0.019574 secs.


### Approach 2

In [81]:
t = time.clock()
qtn2_2_df = df[['From', 'message_type']]
qtn2_2_df = pd.DataFrame(qtn2_2_df[qtn2_2_df['message_type'] == 'broadcast'].groupby(['From'], sort=True).count())
qtn2_2_df = qtn2_2_df.sort_values(['message_type'], ascending=False).head(6)
qtn2_2_df = qtn2_2_df.rename(columns={'message_type': 'Number of emails'})
print qtn2_2_df
print "Completed in: %f secs." % (time.clock()-t)

                          Number of emails
From                                      
j.kaminski@enron.com                     3
steven.kean@enron.com                    2
bg8862@aol.com                           1
michelle.cash@enron.com                  1
mike.mcconnell@enron.com                 1
sarah.novosel@enron.com                  1
Completed in: 0.023442 secs.


## Starting question 3.

In [69]:
t = time.clock()
sel_df = pd.read_sql_query(
        'SELECT D2."Message-ID", d1.Subject, d1."From" as Sent_From, d2."From" as Reply_From, '
        'd1.Date as Sent_date, d2.Date as Reply_date,'
        'Cast((julianday(d2.Date)- julianday(d1.Date))*24*60*60 as Integer) as \'DateDiff\' '
        'FROM processed_data d1, processed_data d2 '
        'where instr(d2.effective_to, d1."From") > 0 and '
        'd2.Subject like d1.Subject and '
        'd1.Date < d2.Date and '
        'd1.Subject not like \'\' and '
        'd1.Subject not like \'FW:\' and '
        'd2.Subject not like \'\' '
        'order by DateDiff limit 5', disk_engine
    )
print sel_df
print "Completed in: %f secs." % (time.clock()-t)

                                      Message-ID  \
0   <4496770.1075842999738.JavaMail.evans@thyme>   
1  <21343473.1075853118912.JavaMail.evans@thyme>   
2   <2612882.1075843476998.JavaMail.evans@thyme>   
3  <26937321.1075843427227.JavaMail.evans@thyme>   
4  <19096180.1075853121576.JavaMail.evans@thyme>   

                                             Subject  \
0  Fwd: Portland Consultant's Investigation Finds...   
1                       CONFIDENTIAL Personnel issue   
2                                         Eeegads...   
3                   CONFIDENTIAL - Residential in CA   
4                       CONFIDENTIAL Personnel issue   

                   Sent_From                 Reply_From            Sent_date  \
0        mary.hain@enron.com     rcarroll@bracepatt.com  2000-10-17 02:04:00   
1    michelle.cash@enron.com  lizzette.palmer@enron.com  2001-10-26 09:18:58   
2     paul.kaufman@enron.com    jeff.dasovich@enron.com  2001-05-10 06:51:00   
3      karen.denne@enron.com  

# Thank you for this great interview. Looking forward to your feedback.